In [ ]:
%matplotlib inline
import os
os.environ["THEANO_FLAGS"] = "device=gpu"
import theano
from caffezoo.googlenet import GoogleNet
from caffezoo.vgg import VGG
from skimage.io import imread
from skimage.transform import AffineTransform, resize, SimilarityTransform, warp
import matplotlib.pyplot as plt
import numpy as np
from sklearn.decomposition import PCA
from sklearn.base import clone
from sklearn.manifold import TSNE, LocallyLinearEmbedding, Isomap
from skimage.util import pad
from sklearn.cross_validation import cross_val_score

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import explained_variance_score
from sklearn.ensemble import RandomForestRegressor
def predictability_score(model, X, y):
    s = 0
    y_hat = model.predict(X)
    for i in range(y.shape[1]):
        s += explained_variance_score(y[:, i], y_hat[:, i])
    return s / y.shape[1]

def predictability(model, X, Y, cv=5):
    if len(X.shape) > 2:
        X = X.reshape((X.shape[0], -1))
    return cross_val_score(model, X, Y, predictability_score, cv=cv)

In [ ]:
import glob
import os
from skimage.io import imread
images_index = []
images = []
for filename in glob.glob("pose/lsp_dataset/images/*.jpg"):
    basename = os.path.basename(filename)
    image_id = int(basename[2:6]) - 1
    images.append(resize(imread(filename), (224, 224)))
    images_index.append(image_id)

images = np.array(images)
images_index = np.array(images_index)

from scipy.io import loadmat
joints = loadmat("pose/lsp_dataset/joints.mat")["joints"]
joints = joints.transpose((2, 0, 1))
joints = joints.reshape((joints.shape[0], -1))
joints = joints[images_index]

In [ ]:
def concat(layers):
    layers = [layer.reshape((layer.shape[0], np.prod(layer.shape[1:]))) for layer in layers]
    return np.concatenate(layers, axis=1)

def gram_matrix(x):
    #x = x.flatten(ndim=3)
    x = x.reshape((x.shape[0], x.shape[1], -1))
    g = np.tensordot(x, x, axes=([2], [2]))
    return g

def reduce_dim(f, dropout=0.5, seed=1234):
    def f_(layers):
        rng = np.random.RandomState(seed)
        layers = [
            l[:, (rng.uniform(size=l.shape[1]) > dropout)] for l in layers
        ]
        return f(layers)
    return f_
def gram(layers):
    layers = [
        gram_matrix(layer) for layer in layers
    ]
    return concat(layers)
def as_is(layers):
    return layers[0]

In [ ]:
L = [ "input",
      "conv2/3x3_reduce",
     "inception_3a/output", "inception_3b/output", 
     "inception_4a/output", "inception_4b/output",
     "inception_4c/output", "inception_4d/output",
     "inception_4e/output", "inception_5a/output",
     "inception_5b/output", "inception_5b/output", "pool5/7x7_s1"]
layers = [L[5]]
model = GoogleNet(resize=False, 
                  layer_names=layers, 
                  aggregate_function=as_is, 
                  batch_size=50)
model._load()

In [ ]:
h = model.transform(images)

In [ ]:
print(h.shape)
print(joints.shape)

In [ ]:
from sklearn.decomposition import PCA, RandomizedPCA
h_ = RandomizedPCA(n_components=100).fit_transform(h.reshape((h.shape[0], -1)))

In [ ]:
predictability(RandomForestRegressor(), h_ , joints)